# Julia Workshop: Function Approximation

## @ CEF 2017

**Authors**: Chase Coleman and Spencer Lyon

**Date**: 27 June 2017


## Goal

There are two main interpolation packages that economists use in Julia: `BasisMatrices.jl` and `Interpolations.jl`.

* [BasisMatrices.jl](https://github.com/QuantEcon/BasisMatrices.jl)
  - Evaluates an approximated function at one (or many) points by building up the corresponding basis matrices
  - Many different types of interpolation (Chebyshev, complete, B-Splines, ...)
  - Portions of this code are based on the [CompEcon](http://www4.ncsu.edu/~pfackler/compecon/toolbox.html) package of Miranda and Fackler, so it should feel familiar for those who have previously used CompEcon.
* [Interpolations.jl](https://github.com/JuliaMath/Interpolations.jl)
  - Never directly stores a basis matrix
  - Only does B-splines (can mix and match degrees of approximation and allows one to choose boundary conditions)
  - Much of this code was written by people who work in image processing. There are lots of tricks that make it hard to beat performance wise.
  - Straightforward to evaluate an interpolator once you have it (overload of indexing methods).
  
This notebook should help you become familiar with what the two packages can do.

## BasisMatrices.jl

In [ ]:
# Pkg.add("BasisMatrices")

In [ ]:
using BasisMatrices

### Basic interpolation

`BasisMatrices` has a type called `Interpoland` which automates large portions of the behind the scenes details.

We show how to build and work with an `Interpoland` below:

#### Step 1: Build the initial grids

We call these the initial grids because certain interpolation types will add additional nodes (in particular, splines often need to add additional knots in order to ensure continuity of derivatives etc...).

In [ ]:
xgrid0 = linspace(0.0, 2.0, 25)
ygrid0 = linspace(-1.0, 1.0, 10);

#### Step 2: Describe desired basis

In [ ]:
xparams = SplineParams(xgrid0, 0, 2)  # Quadratic
yparams = SplineParams(ygrid0, 0, 3)  # Cubic

basis = Basis(xparams, yparams)

# Could also do:
# a_basis = Basis(aparams)
# y_basis = Basis(yparams)
# basis = Basis(a_basis, y_basis)

#### Step 3: Evaluate the function on the basis's nodes

We get the grid of points (including the points added by the package)

$$S = \begin{bmatrix} x_1 & y_1 \\ x_1 & y_2 \\ x_1 & \vdots \\ \vdots & \vdots \\ x_i & y_1 \\ x_i & \vdots \\ \vdots & \vdots \\ x_N & y_N \end{bmatrix}$$

Then evaluate the function $\sin(x) exp(y)$

In [ ]:
S, (xgrid, ygrid) = nodes(basis)

f_vals = sin.(S[:, 1]) .* exp.(S[:, 2]);

#### Step 4: Create interpoland

In [ ]:
itp1 = Interpoland(basis, f_vals);

#### Step 5: Evaluate interpoland

Single point

In [ ]:
eval_1_pt = itp1([0.5, 0.75])
println(eval_1_pt - (sin(0.5)*exp(0.75)))

Many points

In [ ]:
points = [0.0 -0.75
          0.5 -0.5
          0.75 0.25
          0.9 -.35]

eval_many_pt = itp1(points)
println(eval_many_pt - (sin.(points[:, 1]).*exp.(points[:, 2])))

#### Step 6: Update coefficients (if needed)

In [ ]:
func2(S) = S[:, 1].^2 .* S[:, 2]
f_vals2 = func2(S)

update_coefs!(itp1, f_vals2);

println(itp1([1.5, 0.25]) - [sin(1.5)*exp(0.25), 1.5^2 * 0.25])

#### Note: Evaluate multiple functions at once

In [ ]:
itp2 = Interpoland(basis, [f_vals f_vals2])

itp2(points)

#### Note: Derivatives if necessary

In [ ]:
itp1(points, [0 1])  # Take derivative with respect to y

In [ ]:
itp1(points, [0 2])  # 2nd derivative with respect to y

In [ ]:
itp1(points, [1 1])  # Cross partial wrt x and y

### Additional Information

### Directly building basis matrices

Sometimes, understanding how to build basis matrices on your own can speed up an algorithm.

We describe how one might do this. Begin by building up the initial grids and the basis type as in the previous examples.

In [ ]:
xgrid0 = linspace(0.0, 5.0, 25)
ygrid0 = linspace(-2, 2, 10);

xparams = SplineParams(xgrid0, 0, 1)  # Linear
yparams = SplineParams(ygrid0, 0, 3)  # Cubic

basis = Basis(xparams, yparams)


In [ ]:
BasisMatrix(basis, S).vals

### Complete polynomials

No nice type for interpolating with complete polynomials (yet), but can build basis matrices and evaluate.

In [ ]:
_xgrid = repeat(xgrid0, inner=[length(ygrid0), 1])
_ygrid = repeat(ygrid0, outer=[length(xgrid0), 1])
grid = [_xgrid _ygrid]

ϕ = complete_polynomial(grid, 3)

In [ ]:
f_vals3 = grid[:, 1].^2 .* grid[:, 2]
c = ϕ \ f_vals3

In [ ]:
dot(complete_polynomial([0.5, 0.5], 3), c)

In [ ]:
itp2([0.5, 0.5])[2]

### Smolyak Interpolation

Also no nice type for interpolating with complete polynomials (yet), but can still build basis matrices and evaluate.

In [ ]:
# first build parameters
sp = SmolyakParams(2, 3, fill(-1.0, 2), fill(3.0, 2))

In [ ]:
# build the smolyak grid
smol_grid = nodes(sp)

In [ ]:
using PlotlyJS
# set some plotting defaults
use_style!(Style(
    global_trace=attr(mode="markers"), 
    trace=Dict(:scatter3d=>attr(marker_size=5.5), :scatter=>attr(marker_size=8))
))
plot(smol_grid[:, 1], smol_grid[:, 2])

In [ ]:
anisotropic_grid = nodes(SmolyakParams(2, [3, 5], [-1.0, -1.0], [3.0, 3.0]))
plot(anisotropic_grid[:, 1], anisotropic_grid[:, 2])

In [ ]:
f_vals_smol = func2(smol_grid)

# get coefficient vector
c_smol = evalbase(sp, smol_grid) \ f_vals_smol

In [ ]:
# [-1.0,-1.0] × [3.0,3.0]
other_points = rand(40, 2) .*[4 4] .- [1 1]

In [ ]:
trace_grid = scatter(x=smol_grid[:, 1], y=smol_grid[:, 2], name="smolyak grid")
trace_other = scatter(x=other_points[:, 1], y=other_points[:, 2], name="test points")
plot([trace_grid, trace_other])

In [ ]:
plot([
    scatter3d(x=smol_grid[:, 1], y=smol_grid[:, 2], z=f_vals_smol, name="function values"),
    scatter3d(x=smol_grid[:, 1], y=smol_grid[:, 2], z=zeros(size(smol_grid, 1)), name="smolyak grid"),
    scatter3d(x=other_points[:, 1], y=other_points[:, 2], z=func2(other_points), name="wanted"),
])

In [ ]:
smol_approx = evalbase(sp, other_points) * c_smol
wanted = func2(other_points)
maximum(abs, smol_approx - wanted)

## Interpolations.jl

In [ ]:
# Pkg.add("Interpolations")

In [ ]:
using Interpolations

### Interpolating with `Interpolations.jl`

`Interpolations.jl` is light in terms of memory usage because it never actually stores the basis matrices; instead, it computes each element on the fly.

#### Step 1: Build grids and evaluate function on grid

In [ ]:
xgrid = linspace(0.0, 5.0, 25)
ygrid = linspace(-2.0, 2.0, 10);

f_vals = xgrid.^2 .* ygrid'  # Generates a 25x10 matrix

#### Step 2: Create interpolator object

Notice that we don't give it the grids that it is on. By default `Interpolations.jl` pretends all grids are equally spaced with a distance of 1 between them -- Here it would think that the x dimension goes between 1 and 25 and the y dimension goes between 1 and 10.

In [ ]:
itp4 = interpolate(f_vals, BSpline(Cubic(Natural())), OnGrid());

# Can have different degrees of approximation if wanted using below code
# itp = interpolate(f_vals, (BSpline(Linear()), BSpline(Cubic(Natural()))),
#                   OnGrid());

Bottom right element of `f_vals`

In [ ]:
itp4[25, 10]

#### Step 3: Scale the interpolator object

We want to move the grid back to the scales we have. In order to extract extra speed, it is assumed everything is equally spaced. To help enforce this, they require that whatever it is scaled by is a `linspace` of some sort.

In [ ]:
itp4_s = scale(itp4, collect(xgrid), collect(ygrid))  # Won't work

In [ ]:
itp4_s = scale(itp4, xgrid, ygrid);  # Will work

In [ ]:
itp4_s[5.0, 2.0]  # Bottom right element again

#### Step 4: Evaluate

`Interpolations.jl` only allows for evaluation at one point at a time.

In [ ]:
f_interp_values = Array{Float64}(100, 100)
x_fine_grid = linspace(xgrid[1], xgrid[end], 100)
y_fine_grid = linspace(ygrid[1], ygrid[end], 100)

for (ix, x) in enumerate(x_fine_grid)
    for (iy, y) in enumerate(y_fine_grid)
        f_interp_values[ix, iy] = itp4_s[x, y]
    end
end

f_true_values = x_fine_grid.^2 .* y_fine_grid'

println(maximum(abs, f_interp_values - f_true_values))

#### Note: Gradient if needed

In [ ]:
gradient(itp4_s, 1.0, 2.0)